In [2]:
link = "https://www.gutenberg.org/files/100/100-0.txt"
f = requests.get(link)
f.encoding = f.apparent_encoding
lines = f.text
lines = lines.split(
"VENUS AND ADONIS", 1)[1].split(
"*** END OF THE PROJECT GUTENBERG", 1)[0][:500_000]
lines = lines.lower() 
t = " "
f = open('sonnets.txt', 'a')
for sample in lines.split('\n'):
    if sample != '\r':
        if re.match('[0-9 ]+', sample) == None: # sort out the lines that only contains 
                f.write(sample)
f.close()

In [147]:
import torch 
from torch.utils.data import DataLoader
import requests
import spacy
import re
from torchtext.vocab import build_vocab_from_iterator
import torchtext.transforms as T
import torchdata.datapipes as dp
import urllib.request
from bs4 import BeautifulSoup

see
https://betterprogramming.pub/intro-to-rnn-character-level-text-generation-with-pytorch-db02d7e18d89
https://pytorch.org/tutorials/beginner/torchtext_custom_dataset_tutorial.html

In [105]:
eng = spacy.load("en_core_web_sm")
def engTokenize(text):
    """
    Tokenize an English text and return a list of tokens
    """
    return [token.text for token in eng.tokenizer(text)]

def getTokens(data_iter):
    """
    Function to yield tokens from an iterator
    """
    for eng in data_iter:
        if len(eng) != 0:
            if re.match('[0-9 ]+', eng[0]) == None: # sort out the lines that only contains spaces and numbers
                yield engTokenize(eng[0])
def getTransform(vocab):
    """
    Create transforms based on given vocabulary. The returned transform is applied to sequence
    of tokens.
    """
    text_tranform = T.Sequential(
        ## converts the sentences to indices based on given vocabulary
        T.VocabTransform(vocab=vocab),
        ## Add <sos> at beginning of each sentence. 1 because the index for <sos> in vocabulary is
        # 1 as seen in previous section
        T.AddToken(1, begin=True),
        ## Add <eos> at beginning of each sentence. 2 because the index for <eos> in vocabulary is
        # 2 as seen in previous section
        T.AddToken(2, begin=False)
    )
    return text_tranform

['Have', 'a', 'good', 'day', '!', '!', '!']


In [106]:
FILE_PATH = 'sonnets.txt'
data_pipe = dp.iter.IterableWrapper([FILE_PATH])
data_pipe = dp.iter.FileOpener(data_pipe, mode='r')
data_pipe = data_pipe.parse_csv(skip_lines=0, delimiter='\r', as_tuple=False)
for sample in data_pipe:
    print(engTokenize(sample[0]))
    break

['the', 'sonnets']


In [107]:
source_vocab = build_vocab_from_iterator(
    getTokens(data_pipe),
    min_freq=2,
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
    special_first=True
)
'''
<sos> for start of sentence

<eos> for end of sentence

<unk> for unknown words. An example of unknown word is the one skipped because of min_freq=2.

<pad> is the padding token. 
'''
source_vocab.set_default_index(source_vocab['<unk>']) # setting <unk> instead of that unknown word
print(source_vocab.get_itos()[:30])

['<pad>', '<sos>', '<eos>', '<unk>', '.', ',', 'the', 'i', 'and', 'to', 'of', 'you', 'a', 'my', 'that', 'in', 'not', 'is', ';', '’s', '?', 'me', 'it', 'he', 'for', 'his', 'but', 'with', 'be', 'have']


In [108]:
temp_list = list(data_pipe)
some_sentence = temp_list[1][0]
print("Some sentence=", end="")
print(some_sentence)
transformed_sentence = getTransform(source_vocab)(engTokenize(some_sentence))
print("Transformed sentence=", end="")
print(transformed_sentence)
index_to_string = source_vocab.get_itos()
for index in transformed_sentence:
    print(index_to_string[index], end=" ")

Some sentence=from fairest creatures we desire increase,
Transformed sentence=[1, 63, 1476, 1448, 51, 429, 1046, 5, 2]
<sos> from fairest creatures we desire increase , <eos> 

In [109]:
engTokenize(some_sentence)[:-1]

['from', 'fairest', 'creatures', 'we', 'desire', 'increase']

In [110]:
engTokenize(some_sentence)[1:]

['fairest', 'creatures', 'we', 'desire', 'increase', ',']

In [111]:
def applyTransform(sequence):
    """
    Apply transforms to sequence of tokens and create input & target vectors
    """
    tokenized = engTokenize(sequence[0])

    return getTransform(source_vocab)(tokenized)

        
data_pipe = data_pipe.map(applyTransform) ## Apply the function to each element in the iterator

In [113]:
i = 0
for sample in data_pipe:
    print("\n\nInput sentence=", end="")
    index_to_string = source_vocab.get_itos()
    for index in sample:
        print(index_to_string[index], end=" ")
    print("Transformed input sentence=", end="")
    print(sample)
    if i > 1:
        break
    i += 1



Input sentence=<sos> the <unk> <eos> Transformed input sentence=[1, 6, 3, 2]


Input sentence=<sos> from fairest creatures we desire increase , <eos> Transformed input sentence=[1, 63, 1476, 1448, 51, 429, 1046, 5, 2]


Input sentence=<sos> that thereby beauty ’s rose might never die , <eos> Transformed input sentence=[1, 14, 3650, 191, 19, 818, 165, 147, 252, 5, 2]


In [131]:
data_pipe = list(data_pipe)
print(data_pipe[:3])
x = torch.Tensor([item for sublist in data_pipe for item in sublist])
x[:10]

[[1, 6, 3, 2], [1, 63, 1476, 1448, 51, 429, 1046, 5, 2], [1, 14, 3650, 191, 19, 818, 165, 147, 252, 5, 2]]


tensor([1.0000e+00, 6.0000e+00, 3.0000e+00, 2.0000e+00, 1.0000e+00, 6.3000e+01,
        1.4760e+03, 1.4480e+03, 5.1000e+01, 4.2900e+02])

In [118]:
class ShakespeareDataSet(torch.utils.data.Dataset):
    def __init__(
            self, split: 'val' or 'train', vocab_size: int=30_000
            ):
        super(ShakespeareDataSet, self).__init__()
        self.split = split
        self.vocab_size = vocab_size
        self.eng = spacy.load("en_core_web_sm") # Load the English model to tokenize English text
        self.prepare_data()
        self.configure_dataloader()

    def __len__(self):
        return self._x.shape[0]

    def __getitem__(self, index):
        x = self._x[index, :]
        return x
    
    
    def engTokenize(self, text):
        """
        Tokenize an English text and return a list of tokens
        """
        return [token.text for token in self.eng.tokenizer(text)]

    def getTokens(self, data_iter):
        """
        Function to yield tokens from an iterator
        """
        for eng in data_iter:
            if len(eng) != 0:
                if re.match('[0-9 ]+', eng[0]) == None: # sort out the lines that only contains spaces and numbers
                    yield self.engTokenize(eng[0])
    
    def getTransform(self, vocab):
        """
        Create transforms based on given vocabulary. The returned transform is applied to sequence
        of tokens.
        """
        text_tranform = T.Sequential(
            ## converts the sentences to indices based on given vocabulary
            T.VocabTransform(vocab=vocab),
            ## Add <sos> at beginning of each sentence. 1 because the index for <sos> in vocabulary is
            # 1 as seen in previous section
            T.AddToken(1, begin=True),
            ## Add <eos> at beginning of each sentence. 2 because the index for <eos> in vocabulary is
            # 2 as seen in previous section
            T.AddToken(2, begin=False)
        )
        return text_tranform
    
    def applyTransform(sequence):
        """
        Apply transforms to sequence of tokens and create input & target vectors
        """
        tokenized = self.engTokenize(sequence[0])

        return self.getTransform(self.source_vocab)(tokenized)

    def prepare_data(self):
        FILE_PATH = 'sonnets.txt'
        data_pipe = dp.iter.IterableWrapper([FILE_PATH])
        data_pipe = dp.iter.FileOpener(data_pipe, mode='r')
        data_pipe = data_pipe.parse_csv(skip_lines=0, delimiter='\r', as_tuple=False)
        self.source_vocab = build_vocab_from_iterator(
            getTokens(data_pipe),
            min_freq=2,
            specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
            special_first=True
        )
        self.source_vocab.set_default_index(source_vocab['<unk>']) # setting <unk> instead of that unknown word
        data_pipe = data_pipe.map(applyTransform) ## Apply the function to each element in the iterator
        data_pipe = list(data_pipe)
        self._x = torch.Tensor([item for sublist in data_pipe for item in sublist])
        
    def configure_dataloader(self):
        return 
   

In [119]:
ShakespeareDataSet('train')